In [1]:
KAKAO_API_KEY = '541ec48275484cd660aa0b2cf4e80b36' # Rest Api Key( 개인별로 다름 )
API_URL = 'https://dapi.kakao.com/v2/local/search/keyword.json'

In [2]:
import urllib.request # 없으면 설치해야 한다. pip(pip3) install urllib 
import json

In [4]:
import pandas as pd
import numpy as np

In [5]:
crime_anal_police = pd.read_csv('./data/02. crime_in_Seoul.csv', thousands=',', encoding='euc-kr')
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


# 경찰서가 무슨 구에 있나?
* 중부서 -> 중구
* 종로서 -> 종로구

## 검색어 설정을 위한 문자열 편집
* 중부서 -> 서울중부경찰서
* 종로서 -> 서울종로경찰서
* 남대문서 -> 서울남대문경찰서

In [9]:
station_name = []

for name in crime_anal_police['관서명']:
    # print('서울{}경찰서'.format(name[ : -1]))
    station_name.append('서울{}경찰서'.format(name[ : -1]))

station_name

['서울중부경찰서',
 '서울종로경찰서',
 '서울남대문경찰서',
 '서울서대문경찰서',
 '서울혜화경찰서',
 '서울용산경찰서',
 '서울성북경찰서',
 '서울동대문경찰서',
 '서울마포경찰서',
 '서울영등포경찰서',
 '서울성동경찰서',
 '서울동작경찰서',
 '서울광진경찰서',
 '서울서부경찰서',
 '서울강북경찰서',
 '서울금천경찰서',
 '서울중랑경찰서',
 '서울강남경찰서',
 '서울관악경찰서',
 '서울강서경찰서',
 '서울강동경찰서',
 '서울종암경찰서',
 '서울구로경찰서',
 '서울서초경찰서',
 '서울양천경찰서',
 '서울송파경찰서',
 '서울노원경찰서',
 '서울방배경찰서',
 '서울은평경찰서',
 '서울도봉경찰서',
 '서울수서경찰서']

## 카카오 api 에 검색어를 전달하기 위한 검색어 인코딩 작업

kakao api에 전달 가능한 형태로 문자열 인코딩 하기
* urllib.parse.quote 함수를 활용한다.

In [10]:
encText = urllib.parse.quote('서울광진경찰서')
encText

'%EC%84%9C%EC%9A%B8%EA%B4%91%EC%A7%84%EA%B2%BD%EC%B0%B0%EC%84%9C'

In [29]:
request = urllib.request.Request(API_URL+"?query={}".format(encText)) # 카카오 API에 검색까지 포함된 요청을 할 수 있는 요청객체 생성

# kakao에 요청 시에 API_KEY 를 넣지 않으면 인증이 안된 사용자라는 메세지가 뜬다.( UnAuthorization 메세지가 뜬다.)
request.add_header("Authorization", "KakaoAK {}".format(KAKAO_API_KEY)) # 요청시 카카오 API에 나의 인증정보를 헤더에 전달

response = urllib.request.urlopen(request) # 요청 수행 후 카카오 서버로부터 응답 받기

# read() 요청된 내용을 모두 불러오는 역할 -> 한번 불러오면 다시 불러 올 수 없다. (다시 요청 해야 한다.)
# json.loads(response.read()) # 응답받은 내용 확인 ( json 형태로 변환했기 때문에, dictionary 형태로 사용할 수 있다. )

### 필요 데이터 추출
# 1. 경찰서 주소
# 2. 위, 경도
response_json = json.loads(response.read().decode('utf-8')) 
temp_data = response_json['documents'][0]

temp_data

{'address_name': '서울 광진구 구의동 254-32',
 'category_group_code': 'PO3',
 'category_group_name': '공공기관',
 'category_name': '사회,공공기관 > 행정기관 > 경찰서',
 'distance': '',
 'id': '8211607',
 'phone': '02-2285-7602',
 'place_name': '서울광진경찰서',
 'place_url': 'http://place.map.kakao.com/8211607',
 'road_address_name': '서울 광진구 자양로 167',
 'x': '127.08396170505674',
 'y': '37.54292164557882'}

In [30]:
print(temp_data['address_name'])
print(temp_data['y']) # 경도
print(temp_data['x']) # 위도

서울 광진구 구의동 254-32
37.54292164557882
127.08396170505674


# 전체 경찰서에 대한 주소 및 위,경도 모으기

In [48]:
station_address = [] # 경찰서의 주소를 모을 리스트
station_lat = [] # 위도 -> x
station_lon = [] # 경도 -> y

In [49]:
for name in station_name:
    encText = urllib.parse.quote(name)
    request = urllib.request.Request(API_URL+"?query={}".format(encText)) # 카카오 API에 검색까지 포함된 요청을 할 수 있는 요청객체 생성
    request.add_header("Authorization", "KakaoAK {}".format(KAKAO_API_KEY)) 

    response = urllib.request.urlopen(request) # 요청 수행 후 카카오 서버로부터 응답 받기
    
    # 응답 코드 받기 OK : 200, 기타 응답 오류 코드를 수신할 수 있다.
    rescode = response.getcode() 
    
    # rescode가 200이면 성공적으로 응답 받았다.
    if rescode == 200 :
        # 응답을 받고, 그 응답에 대한 데이터를 보통 body 라고 이야기 한다.
        response_body = response.read()
        
        # 응답받은 데이터(response_body)를 utf-8로 인코딩한 후 json 객체로 만들어 주면
        tmp = json.loads(response_body.decode('utf-8'))
        tmp_doc = tmp['documents']
        tmp_data = tmp_doc[0]
        
        address = tmp_data['address_name']
        lat = tmp_data['x']
        lng = tmp_data['y']
        
        station_address.append(address) # 리스트에 주소 추가
        station_lat.append(lat) # 리스트에 경도 추가
        station_lon.append(lng) # 리스트에 위도 추가
        
        print("{} ---> {}".format(name, address))
        
    

서울중부경찰서 ---> 서울 중구 저동2가 62-1
서울종로경찰서 ---> 서울 종로구 경운동 90-18
서울남대문경찰서 ---> 서울 중구 남대문로5가 567
서울서대문경찰서 ---> 서울 서대문구 미근동 165
서울혜화경찰서 ---> 서울 종로구 인의동 48-57
서울용산경찰서 ---> 서울 용산구 원효로1가 12-12
서울성북경찰서 ---> 서울 성북구 삼선동5가 301
서울동대문경찰서 ---> 서울 동대문구 청량리동 229
서울마포경찰서 ---> 서울 마포구 아현동 618-1
서울영등포경찰서 ---> 서울 영등포구 당산동3가 2-11
서울성동경찰서 ---> 서울 성동구 행당동 192-8
서울동작경찰서 ---> 서울 동작구 노량진동 72-35
서울광진경찰서 ---> 서울 광진구 구의동 254-32
서울서부경찰서 ---> 서울 은평구 녹번동 177-15
서울강북경찰서 ---> 서울 강북구 번동 415-15
서울금천경찰서 ---> 서울 금천구 시흥동 1030
서울중랑경찰서 ---> 서울 중랑구 신내동 810
서울강남경찰서 ---> 서울 강남구 대치동 998
서울관악경찰서 ---> 서울 관악구 봉천동 1695-5
서울강서경찰서 ---> 서울 양천구 신월동 25
서울강동경찰서 ---> 서울 강동구 성내동 541-1
서울종암경찰서 ---> 서울 성북구 종암동 3-1260
서울구로경찰서 ---> 서울 구로구 구로동 436
서울서초경찰서 ---> 서울 서초구 서초동 1726-1
서울양천경찰서 ---> 서울 양천구 신정동 321
서울송파경찰서 ---> 서울 송파구 가락동 9
서울노원경찰서 ---> 서울 노원구 하계동 250
서울방배경찰서 ---> 서울 서초구 방배동 455-10
서울은평경찰서 ---> 서울 은평구 불광동 산 24
서울도봉경찰서 ---> 서울 도봉구 창동 17
서울수서경찰서 ---> 서울 강남구 개포동 14


In [50]:
station_address[:5]

['서울 중구 저동2가 62-1',
 '서울 종로구 경운동 90-18',
 '서울 중구 남대문로5가 567',
 '서울 서대문구 미근동 165',
 '서울 종로구 인의동 48-57']

In [51]:
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


# 경찰서 위치에 대한 구별 데이터 추가

* split() 으로 문자열을 짜르고 인덱스 1번을 ~구로 삼게 되면
 * 서울특별시 ~구 ( O )
 * 서울 ~구 ( O )
 * 서울 특별시 ~구 ( X )

* split('구')로 자르고 난 다음 편집
 * 구의 이름만 등장한다. -> 서울 광진구 -> [ 서울 특별시 광진, 가 ] -> split() -> [-1] 쓰면 된다.
 * 문제 발생 상황 : 구로구 ( X )
 
문자열 데이터에서 내가 필요한 정보만 가지고 올 수 있게 하는 것 -> 텍스트 마이닝

우리가 하고 있는건 전처리 알고리즘을 직접 생각하고 나서 할 수 있는 텍스트 마이닝

In [52]:
gu_name = []

for name in station_address:
    tmp = name.split() # 공백을 기준으로 자른다.
    
    # 구로 끝나는 부분만 추출할 수 있는 정규식을 만들어서 사용할 수도 있다.
    tmp_gu = [gu for gu in tmp if gu[-1] == '구'] # 공백을 기준으로 잘린 리스트를 검사해서 끝이 '구'로 끝나는 부분만 리스트에 추가
    print(tmp_gu[0])
    
    gu_name.append(tmp_gu[0]) # 구의 이름 추가

중구
종로구
중구
서대문구
종로구
용산구
성북구
동대문구
마포구
영등포구
성동구
동작구
광진구
은평구
강북구
금천구
중랑구
강남구
관악구
양천구
강동구
성북구
구로구
서초구
양천구
송파구
노원구
서초구
은평구
도봉구
강남구


# 각 경찰서에 대한 구의 추출 후 데이터 프레임에 추가

In [55]:
crime_anal_police['구별'] = gu_name
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,서대문구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,종로구


# 메인 주제 : 구별 범죄 발생, 범죄 검거에 대한 분석

In [56]:
# 구별 데이터가 잘 수집 되었는지 확인이 필요
crime_anal_police['구별'].unique()

array(['중구', '종로구', '서대문구', '용산구', '성북구', '동대문구', '마포구', '영등포구', '성동구',
       '동작구', '광진구', '은평구', '강북구', '금천구', '중랑구', '강남구', '관악구', '양천구',
       '강동구', '구로구', '서초구', '송파구', '노원구', '도봉구'], dtype=object)

In [57]:
len(crime_anal_police['구별'].unique()) # 서울시의 구는 25개 행정구가 존재하는데 실제 수집된건 24개

24

데이터 분석에서는 내가 분석하고자 하는 대상의 비즈니스를 잘 알고 있는 것도 중요하다.

* 데이터 분석 전 자료조사가 필요할 수도 있다.
* 설문조사, 논문, 검색 등을 참고한다. 

### 데이터 분석에서 제일 시간이 많이 걸리는 것 중에 하나가 비즈니스 분석

In [58]:
# 2020-07-13 기준 서울강서경찰서는 양천구에 임시청사가 있다.
crime_anal_police.loc[crime_anal_police['관서명']=='강서서']

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
19,강서서,7,8,13,13,262,191,2096,1260,3207,2718,양천구


In [59]:
crime_anal_police.loc[crime_anal_police['관서명']=='강서서', '구별'] = '강서구'

In [60]:
crime_anal_police

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,서대문구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,종로구
5,용산서,5,5,14,14,194,173,1557,587,2050,1704,용산구
6,성북서,2,2,2,1,86,71,953,409,1194,1015,성북구
7,동대문서,5,5,13,13,173,146,1981,814,2548,2227,동대문구
8,마포서,8,8,14,10,294,247,2555,813,2983,2519,마포구
9,영등포서,14,12,22,20,295,183,2964,978,3572,2961,영등포구


In [62]:
crime_anal_police.to_csv('./data/crime_anal_police_gu.csv', encoding='utf-8')